<a href="https://colab.research.google.com/github/limkaisean/Test/blob/master/Test-inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install necessary packages

In [ ]:
!pip install --user -r requirements.txt

#Imports

In [47]:
import numpy as np
import torch
import dask
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
from IPython.display import Image, display

#Hyperparameters

In [67]:
BATCH_SIZE = 4
NUM_WORKERS = 1
NO_OF_IMAGE = 200

#Load model

In [68]:
import time

start = time.time()

PATH = 'inference-model.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(PATH) if device == "cuda" else torch.load(PATH,map_location ='cpu')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#Load and preprocess data


In [69]:
datadir = 'sundatabasesubset'
def preprocess(datadir, no_of_image=int(NO_OF_IMAGE)):
    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])
    eval_data = datasets.ImageFolder(datadir,       
                    transform=preprocess)
    num_eval = len(eval_data)
    indices = list(range(num_eval))
    np.random.shuffle(indices)
    eval_idx = indices[:no_of_image]
    data_sampler = SubsetRandomSampler(eval_idx)
    dataloader = torch.utils.data.DataLoader(eval_data,
                   sampler=data_sampler, batch_size=int(BATCH_SIZE), num_workers=int(NUM_WORKERS))
    return dataloader

dataloader = preprocess(datadir, int(NO_OF_IMAGE))

#Inferencing

In [70]:
# @dask.delayed
def predict(inputs, device, model):
  inputs = inputs.to(device)
  output = model(inputs)
  return output

def inference(dataloader):
  predicted = []
  with torch.no_grad():
    for inputs, labels in dataloader:
      prediction = predict(inputs, device, model)
      predicted.append(prediction)
  # predicted = dask.compute(*predicted)
  return predicted


In [71]:
predicted = inference(dataloader)
end = time.time()
time_elapsed = end - start

In [72]:
print(time_elapsed)

40.06146788597107
